## 売上資料Ⅱ
## 得意先別大分類別月別売上実績表　作成

- 作成年度と作成月の入力を忘れずに！
- 情報システムが作表しているので使用せず

In [ ]:
import numpy as np
import pandas as pd
import pickle

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def data_read():
    # 売上
    target_col = ['年度','売上月','請求先CD','請求先名1','大分類','小分類','品目','品名','規格','販売数量','販売金額','予算金額']
    uriagedf_i = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/I_U_2009_2021.csv",usecols=target_col)
    uriagedf_i = uriagedf_i[uriagedf_i['請求先CD'] != 80004]
    uriagedf_i = uriagedf_i[uriagedf_i['請求先CD'] != 0]

    # Class、大分類、小分類各dict
    with open('/content/drive/MyDrive/Purchased_products/class_dict.pkl', 'rb') as f:
        class_dict = pickle.load(f)
    with open('/content/drive/MyDrive/Purchased_products/dai_dict.pkl', 'rb') as f:
        dai_dict = pickle.load(f)
    with open('/content/drive/MyDrive/Purchased_products/syo_dict.pkl', 'rb') as f:
        syo_dict = pickle.load(f)

    # Excelにある得意先コードを読込
    tokuisaki_df = pd.read_table("/content/drive/MyDrive/Purchased_products/大分類用得意先コード.txt")

    # list化
    tokuisaki_list = []
    for i in range(len(tokuisaki_df)):
        tokuisaki_list.append(tokuisaki_df.iloc[i]['得意先コード'])
    
    return uriagedf_i, class_dict, dai_dict, syo_dict, tokuisaki_list

In [ ]:
# 作成年度を入力
nendo = 2021
# 作成月を入力（1～12を入力）
month = 8

# Excel上に得意先コードの有無をcheck　OKなら以下を進める
# データ読込
uriagedf_i, class_dict, dai_dict, syo_dict, tokuisaki_list = data_read()

# 得意先コードcheck
uriagedf_i["請求先CD"] = uriagedf_i["請求先CD"].astype(int)
seikyu_cd = uriagedf_i["請求先CD"].unique()
err_cnt = 0
for i in range(len(uriagedf_i)):
    if uriagedf_i.iloc[i]['請求先CD'] not in tokuisaki_list:
        err_cnt += 1
        print('得意先コードがExcelにありません。cd={} {}'.format(uriagedf_i.iloc[i]['請求先CD'],\
                                                                uriagedf_i.iloc[i]['請求先名1']))
if err_cnt == 0:
    print('得意先コードに問題はありません。以下の処理を進めてください')      

得意先コードに問題はありません。以下の処理を進めてください


In [ ]:
def month_set(nendo, month):
    """年と月で6桁の年月にする
    """
    if month < 6:
        this_month = int(str(nendo+1) + "0" + str(month))
    else:
        if month > 9:
            this_month = int(str(nendo) + str(month))
        else:
            this_month = int(str(nendo) + "0" + str(month))
    
    return this_month

In [ ]:
# 前年度を設定
zen_nendo = nendo - 1

# 当月を設定
this_month = month_set(nendo, month)

# 前年当月を設定
zen_month = month_set(zen_nendo, month)

# 当年の最終月を設定
end_month = month_set(nendo, 5)

# 12ヵ月分のlistを作成
list_12 = []
m = 6
y = month_set(nendo, m)
list_12.append(y)
for i in range(11):
    m += 1
    if m > 12:
        m = 1
    y = month_set(nendo, m)
    list_12.append(y)
print('当月：', this_month)
print('前年当月', zen_month)
print('最終月', end_month)
print(list_12)

当月： 202108
前年当月 202008
最終月 202205
[202106, 202107, 202108, 202109, 202110, 202111, 202112, 202201, 202202, 202203, 202204, 202205]


In [ ]:
# 当年度、前年度の棚卸月を設定
tanaorosi_nendo = []
tanaorosi_zen_nendo = []

tmp_month = int(str(nendo) + "0" + str(5))
tanaorosi_nendo.append(tmp_month)
tmp_month = int(str(nendo-1) + "0" + str(5))
tanaorosi_zen_nendo.append(tmp_month)

start_m = 6
tmp_month = month_set(nendo, start_m)

while tmp_month <= this_month:   # 当年度セット
    tanaorosi_nendo.append(tmp_month)
    if tmp_month == this_month:
        break
    
    start_m += 1
    if start_m > 12:
        start_m = 1
    tmp_month = month_set(nendo, start_m)

start_m = 6
tmp_month = month_set(zen_nendo, start_m)

while tmp_month <= zen_month:   # 前年度セット
    tanaorosi_zen_nendo.append(tmp_month)
    if tmp_month == zen_month:
        break
    
    start_m += 1
    if start_m > 12:
        start_m = 1
    tmp_month = month_set(zen_nendo, start_m)

# print
print('nendo= {}'.format(nendo))
print('tanaorosi_nendo= {}'.format(tanaorosi_nendo))
print('tanaorosi_zen_nendo= {}'.format(tanaorosi_zen_nendo))

nendo= 2021
tanaorosi_nendo= [202105, 202106, 202107, 202108]
tanaorosi_zen_nendo= [202005, 202006, 202007, 202008]


In [ ]:
# uriagedf_i 整理
uriagedf_i = uriagedf_i.fillna(0)
uriagedf_i["請求先CD"] = uriagedf_i["請求先CD"].astype(int)
uriagedf_i["売上月"] = uriagedf_i["売上月"].astype(int)
uriagedf_i["販売数量"] = uriagedf_i["販売数量"].astype(int)
uriagedf_i["販売金額"] = uriagedf_i["販売金額"].astype(int)
uriagedf_i["予算金額"] = uriagedf_i["予算金額"].astype(int)
uriagedf_i["年度"] = uriagedf_i["年度"].astype(int)

### データ作成

In [ ]:
# 売上データ絞り込み
uriagedf_i = uriagedf_i[(uriagedf_i["売上月"] >=tanaorosi_zen_nendo[0]) & (uriagedf_i["売上月"] <= tanaorosi_nendo[-1])]

# Class、大分類、小分類各コードをセット
uriagedf_i['大分類CD'] = uriagedf_i['大分類'].str[:4]
uriagedf_i['小分類CD'] = uriagedf_i['小分類'].str[:4]

class_list = []
for i in range(len(uriagedf_i)):
    class_list.append(dai_dict[uriagedf_i.iloc[i]['大分類CD']][1])
uriagedf_i['Classcd'] = class_list

# 月ごと販売金額をdictへ
uriage_dict = dict(uriagedf_i.groupby(["売上月"])["販売金額"].sum())
# 月ごと予算金額をdictへ
yosan_dict = dict(uriagedf_i.groupby(["売上月"])["予算金額"].sum())

In [ ]:
# 請求先dict
seikyu_dict = dict(uriagedf_i.groupby('請求先CD')['請求先名1'].max())

In [ ]:
def zennen_keisan(nen_month):
    nen_month_str = str(nen_month)
    nen_int = int(nen_month_str[:4])
    nen_month_str = str(nen_int - 1) + nen_month_str[4:]

    return int(nen_month_str)

In [ ]:
list_12_z = []
for i in list_12:
    list_12_z.append(zennen_keisan(i))

In [ ]:
# 得意先別大分類別セット
seikyu_dai_df = uriagedf_i.groupby(['請求先CD','Classcd','大分類CD','売上月'],as_index=False).sum()
seikyu_dai_df = seikyu_dai_df.sort_values(['請求先CD','Classcd','大分類CD','売上月'])

seikyu_dai_list = []
for i in range(len(seikyu_dai_df)):
    tmp_list = []
    tmp_save_list = []
    tmp_list.append(seikyu_dai_df.iloc[i]["請求先CD"])
    tmp_list.append(seikyu_dict[seikyu_dai_df.iloc[i]["請求先CD"]])
    tmp_list.append(seikyu_dai_df.iloc[i]["Classcd"])
    tmp_list.append(class_dict[seikyu_dai_df.iloc[i]["Classcd"]])
    tmp_list.append(seikyu_dai_df.iloc[i]["大分類CD"])
    tmp_list.append(dai_dict[seikyu_dai_df.iloc[i]["大分類CD"]][0])
    if seikyu_dai_df.iloc[i]['売上月'] in list_12:
        tmp_list.append(seikyu_dai_df.iloc[i]['売上月'])
        tmp_list.append(list_12.index(seikyu_dai_df.iloc[i]['売上月']))

        tmp_save_list[:] = tmp_list
        tmp_list.append(2) # 予算区分
        tmp_list.append(seikyu_dai_df.iloc[i]["予算金額"])
        seikyu_dai_list.append(tmp_list)

        tmp_save_list.append(3) # 当年区分
        tmp_save_list.append(seikyu_dai_df.iloc[i]["販売金額"])
        seikyu_dai_list.append(tmp_save_list)
        
    elif seikyu_dai_df.iloc[i]['売上月'] in list_12_z:
        tmp_list.append(seikyu_dai_df.iloc[i]['売上月'])
        tmp_list.append(list_12_z.index(seikyu_dai_df.iloc[i]['売上月']))
        tmp_list.append(1) # 前年区分
        tmp_list.append(seikyu_dai_df.iloc[i]["販売金額"])
        seikyu_dai_list.append(tmp_list)
    else:
        continue

In [ ]:
col_nm = ['請求先CD','請求先名','Classcd','Class名','大分類CD','大分類名','売上月','月番号','年区分','金額']

In [ ]:
seikyu_dai_df = pd.DataFrame(seikyu_dai_list, columns=col_nm)

In [ ]:
seikyu_dai_df

In [ ]:
# Excelデータ作成
seikyu_dai_df.to_excel("/content/drive/MyDrive/Purchased_product/得意先別大分類.xlsx", sheet_name='得意先別大分類')